In [ ]:
import os
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import proj3d

from sklearn.preprocessing import StandardScaler

import umap
import umap.plot

import warnings
warnings.filterwarnings('ignore')

from bokeh.plotting import figure, show, output_notebook
from bokeh.models import HoverTool, ColumnDataSource, CategoricalColorMapper
from bokeh.palettes import Spectral10

# 파일경로 설정

In [ ]:
# data_path = os.path.join(os.getcwd(),'data')
# files = os.listdir(data_path)[:-1]

# aisles = pd.read_csv(os.path.join(data_path,files[0]), engine='pyarrow') # (134, 2)
# departments = pd.read_csv(os.path.join(data_path,files[1]), engine='pyarrow') # (21, 2)
# orders = pd.read_csv(os.path.join(data_path,files[2]), engine='pyarrow') # (3421083, 7)
# orders_products_prior = pd.read_csv(os.path.join(data_path,files[3]),engine='pyarrow') # (32434489, 4)
# orders_products_train = pd.read_csv(os.path.join(data_path,files[4]),engine='pyarrow') # (1384617, 4)
# products = pd.read_csv(os.path.join(data_path,files[5]),engine='pyarrow') # (49688, 4)

데이터 설명
* aisles - 통로(코너, 냉동 department의 어류 이런 세부종류?) aisleid, aisle
* departments - department종류(냉동식품 등)  department id, department
* orders - userid, eval_set(prior, train, test), 주문번호, 주문요일?, 주문시간대, since prior order
* order_products_* - orderid, 상품id, add to cart order, reorderd(재주문 여부, none은 0으로)
* products - 상품id, 상품명, aisleid, departmentid

# 분석용 전체 데이터 생성

In [ ]:
# prior = orders[orders['eval_set']== 'prior']
# train = orders[orders['eval_set'] == 'train']
# test = orders[orders['eval_set'] == 'test']
# del train['eval_set'], test['eval_set'], prior['eval_set']

# temp = orders_products_train.set_index('order_id')
# orders_train = pd.merge(temp, train, on='order_id')

# temp = orders_products_prior.set_index('order_id')
# orders_prior = pd.merge(temp, prior, on='order_id')

# total_orders = pd.concat([orders_prior, orders_train])
# temp = pd.merge(total_orders,products , on='product_id')
# temp = pd.merge(temp, aisles, on='aisle_id')
# temp = pd.merge(temp, departments, on='department_id')

# total_orders.to_parquet('total_orders.parquet', index=False)

# 분석

In [ ]:
total_orders = pd.read_parquet('/content/drive/MyDrive/total_orders.parquet',engine='fastparquet')

In [ ]:
total_orders.head()

## 기본 분석

## 군집화
재구매율을 통해 고객 군집화를 통한 고객 segmentation

### user_id 기준(한 사람이 여러번 주문한것을 한번에 처리)

In [ ]:
# 전처리 코드 모음
# temp=total_orders.groupby(['user_id']).agg({'reordered':'sum', 'user_id':'count'}).rename(columns={'user_id':'count'}).reset_index()
# temp['reorder_ratio'] = temp['reordered'] / temp['count']
# temp['class'] = pd.qcut(temp['reorder_ratio'],[0,0.25,0.5,0.75,1], labels=[4,3,2,1])


# tab1 = pd.crosstab(total_orders['user_id'], total_orders['aisle'])
# label_table = temp[['user_id','class']]
# tab1 = pd.merge(tab1,label_table, on='user_id')

# order_list = total_orders.groupby(['user_id']).product_name.apply(list).reset_index(name='order_list').set_index('user_id')['order_list']
# table = pd.merge(tab1, order_list, on='user_id')
# table.to_parquet('table.parquet',index=False)
table = pd.read_parquet('table.parquet',engine='fastparquet')
label1, order_list = table['class'], table['order_list']
tab1 = table.drop(['class', 'order_list'], axis=1)

In [ ]:
temp=total_orders.groupby(['user_id']).agg({'reordered':'sum', 'user_id':'count'}).rename(columns={'user_id':'count'}).reset_index()
temp['reorder_ratio'] = temp['reordered'] / temp['count']
temp.head()


In [ ]:
sns.histplot(temp['reorder_ratio'])

In [ ]:
temp['class'] = pd.qcut(temp['reorder_ratio'],[0,0.25,0.5,0.75,1], labels=[4,3,2,1])

In [ ]:
# 재구매율 기준 
temp['class'].value_counts().plot(kind='bar')

#### aisle(중분류 기준)

In [ ]:
tab1 = pd.crosstab(total_orders['user_id'], total_orders['aisle'])
label_table = temp[['user_id','class']]
label1 = pd.merge(tab1,label_table, on='user_id')['class']

#### department(대분류 기준)

In [ ]:
tab2 = pd.crosstab(total_orders['user_id'], total_orders['department'])
label_table = temp[['user_id','class']]
label2 = pd.merge(tab2,label_table, on='user_id')['class']

### Umap 적용

소분류 사용

In [ ]:
scaled_tab1 = StandardScaler().fit_transform(tab1.drop['user_id'])
embedding1 = umap.UMAP(random_state=42).fit(scaled_tab1) 
                    #   n_neighbors=10,
                    #   min_dist=0.2,
                    #   metric='correlation'# 인자 공부....?

In [ ]:
umap.plot.points(embedding1, label1, theme='fire')
# class, user_id, order_id 구매목록 뜨게하기?

In [ ]:
scaled_tab1 = StandardScaler().fit_transform(tab1)
mapper = umap.UMAP(random_state=42).fit(scaled_tab1[:50000])

In [ ]:
umap.plot.output_notebook()
hover_data = pd.DataFrame({'User':df['user_id'][:50000],
                           'Ordered':df['order_list'][:50000]})


p = umap.plot.interactive(mapper, labels=df['class'][:50000], hover_data=hover_data, point_size=2)
umap.plot.show(p)

bokeh?

In [ ]:
scaled_tab1 = StandardScaler().fit_transform(tab1.drop('user_id', axis=1))
embedding1 = umap.UMAP(random_state=42).fit_transform(scaled_tab1) 

In [ ]:
# df = pd.DataFrame(embedding1, columns = ('x','y'))
# df['user_id'] = tab1['user_id']
# df['class'] = label1
# df['order_list'] = order_list
# df.to_parquet('/content/drive/MyDrive/datasource.parquet',index=False)
df = pd.read_parquet('datasource.parquet',engine='fastparquet')

In [ ]:
scaled_tab1 = StandardScaler().fit_transform(tab1.drop('user_id', axis=1))
embedding1_3 = umap.UMAP(random_state=42, n_components=3).fit_transform(scaled_tab1) 

In [ ]:
# df = pd.DataFrame(embedding1_3, columns = ('x','y','z'))
# df['user_id'] = tab1['user_id']
# df['class'] = label1
# df['order_list'] = order_list
# df.to_parquet('datasource2.parquet',index=False)
df = pd.read_parquet('datasource2.parquet',engine='fastparquet')

In [ ]:
df

In [ ]:
datasource = ColumnDataSource(df)
color_mapping = CategoricalColorMapper(factors=['Class1','Class2','Class3','Class4'],
                                       palette=Spectral10)
plot_figure = figure(
    title='UMAP projection of the user-product',
    plot_width=800,
    plot_height=800,
    tools=('pan, wheel_zoom, reset')
)

plot_figure.add_tools(HoverTool(names=['df'],tooltips="""
<div style="margin: 10">
    <div style="margin: 0 auto; width:300px;">
        <span style="font-size: 12px; font-weight: bold;">User:</span>
        <span style="font-size: 12px">@user_id</span>
        <span style="font-size: 12px; font-weight: bold;">Ordered:</span>
        <span style="font-size: 12px">@order_list</span>
    </div>
</div>
"""))

plot_figure.circle(
    'x',
    'y',
    source=datasource,
    color=dict(field='class', transform=color_mapping),
    line_alpha=0.6,
    fill_alpha=0.6,
    size=4,
    name='df'
)

show(plot_figure)


In [ ]:
fig, ax = plt.subplots(figsize=(10,10))
names = ['Class1','Class2','Class3','Class4']
for i in range(4):
    ax.scatter(df.loc[df['class']==i+1,'x'],
               df.loc[df['class']==i+1,'y'],
               cmap='tab10',marker='o',s=.5,
               alpha=0.5,
               label=names[i])

ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.legend(loc='upper right',markerscale=5)
ax.grid(True)

plt.show()

In [ ]:
fig = plt.figure(figsize=(10, 10))
names = ['Class1','Class2','Class3','Class4']
ax = fig.add_subplot(projection='3d')

for i in range(4):
    ax.scatter(df.loc[df['class']==i+1,'x'],
               df.loc[df['class']==i+1,'y'],
               df.loc[df['class']==i+1,'z'],
               cmap='tab10',marker='o',s=.2,
               alpha=0.5,label=names[i])
    
ax.xaxis.set_ticklabels([])
ax.yaxis.set_ticklabels([])
ax.zaxis.set_ticklabels([])
ax.tick_params(color='0.8')
ax.legend(loc='upper right',markerscale=3)
ax.view_init(30,120)

plt.tight_layout()
plt.show()

중분류 사용

In [ ]:
scaled_tab2 = StandardScaler().fit_transform(tab2)
embedding2 = umap.UMAP(random_state=42).fit(scaled_tab2) 

In [ ]:
umap.plot.points(embedding2, label2, theme='fire')

### order_id 기준(한 사람이 주문해도 다른 주문으로 처리)

In [ ]:
temp=total_orders.groupby(['order_id']).agg({'reordered':'sum', 'user_id':'count'}).rename(columns={'user_id':'count'}).reset_index()
temp['reorder_ratio'] = temp['reordered'] / temp['count']
temp.head()


In [ ]:
sns.histplot(temp['reorder_ratio'])

In [ ]:
# 재구매율 기준 
temp['class'] = pd.qcut(temp['reorder_ratio'],[0,0.25,0.5,0.75,1], labels=[4,3,2,1])
temp['class'].value_counts().plot(kind='bar')

#### aisle(중분류 기준)

In [ ]:
tab1 = pd.crosstab(total_orders['order_id'], total_orders['aisle'])
label_table = temp[['order_id','class']]
label1 = pd.merge(tab1,label_table, on='order_id')['class']

#### department(대분류 기준)

In [ ]:
tab2 = pd.crosstab(total_orders['order_id'], total_orders['department'])
label_table = temp[['order_id','class']]
label2 = pd.merge(tab2,label_table, on='order_id')['class']

### Umap 적용

In [ ]:
scaled_tab1 = StandardScaler().fit_transform(tab1)
embedding1 = umap.UMAP(random_state=42).fit(scaled_tab1) 
                    #   n_neighbors=10,
                    #   min_dist=0.2,
                    #   metric='correlation'# 인자 공부....?

In [ ]:
umap.plot.points(embedding1, label1, theme='fire')

In [ ]:
scaled_tab2 = StandardScaler().fit_transform(tab2)
embedding2 = umap.UMAP(random_state=42).fit(scaled_tab2) 

In [ ]:
umap.plot.points(embedding2, label2, theme='fire')

In [ ]:
labels = HDBSCAN(
    min_samples=100,
    min_cluster_size=1000
).fit_predict(X_reduced) # 또는 임베딩을 군집화해서 라벨링

mapper = UMAP(
      random_state=42,
      metric='euclidean',
      n_components=2,
      n_neighbors= 24,
      min_dist=0.1).fit(X.drop('label', axis=1))